In [ ]:
cur_el = joblib.load('it.sav')

In [ ]:
#cur_el = 1

In [ ]:
base_folder = Path('C:/Users/ilkele4s/')

main_results_folder = Path('ML_Results/LSTM_stratified_singleElectrode_results')
new_results_folder = 'electrode_' +str(cur_el)

data_folder = Path('Data/Electrodes')
data_file = 'electrode_' +str(cur_el) + '.csv'

tune_folder = 'Tune'
project_name = 'Random_30'

In [ ]:
results_dir = os.path.join(base_folder,main_results_folder)

#if cur_el == 1:
#    os.mkdir(results_dir)

new_results_dir = os.path.join(results_dir,new_results_folder)

In [ ]:
os.mkdir(new_results_dir)

print(new_results_dir)

In [ ]:
tune_dir = os.path.join(new_results_dir,tune_folder)

In [ ]:
# load the data 
data_dir = os.path.join(base_folder,data_folder)

data_file_path = os.path.join(data_dir,data_file)

print(data_file_path)

Data = pd.read_csv(data_file_path, header = None)

In [ ]:
# structure samples for LSTM

data_array = np.array(Data)

n_samples = data_array.shape[0]
n_features = data_array.shape[1]

bin_cur = 1
bin_past = 3

bins_predict = bin_cur + bin_past # 3 previous + 1 current

formatted_data = np.zeros([n_samples,bins_predict,n_features])

start_idx=0
for i in range(n_samples-bin_past):
    end_idx=start_idx+bins_predict # indices for 4 elements (sliding fashion)
    formatted_data[i+bin_past,:,:]=data_array[start_idx:end_idx,:] # indexing the data with indices from the previous step
    start_idx=start_idx+1

In [ ]:
formatted_data[np.isnan(formatted_data)] = 0 # replacing nans with zeros

In [ ]:
# create a label set

n_zones = 45
n_trials = 32

## one-hot-encoding 
cat_zones = keras.utils.to_categorical(np.r_[1:n_zones+1] ,num_classes=46) # one-hot-encoding 
cat_zones = np.delete(cat_zones,0,1)

## repeating the labels for 32 (no_of trial) times
labels_all_trials = np.tile(cat_zones, 32) 


## reshaping the data from  (45,1440) to (1440,45)
reshaped_labels_all_trials = np.reshape(labels_all_trials, [n_samples, 45])



In [ ]:
# first train - test split -- STRATIFIED

## later train will be splitted into train - validation
## we aim for 60 - 20 - 20 

train_samples, test_samples, train_labels, test_labels = train_test_split(formatted_data, reshaped_labels_all_trials,
                                                                          test_size=.20, 
                                                                          stratify=reshaped_labels_all_trials, 
                                                                          random_state=42)

In [ ]:
## calculate the split size for the second split to achive aimed ratios

train_size = np.shape(train_samples)
test_size = np.shape(test_samples)
split_size =1/(train_size[0]/ test_size[0])
#print(split_size)

In [ ]:
# train - validation split -- STRATIFIED

train_samples, validation_samples, train_labels, validation_labels, = train_test_split(train_samples, train_labels, 
                                                                                       test_size=.25, 
                                                                                       stratify=train_labels, 
                                                                                       random_state=42)

In [ ]:
joblib.dump(train_samples, os.path.join(new_results_dir,'train_samples.sav'))
joblib.dump(train_labels, os.path.join(new_results_dir,'train_labels.sav'))

joblib.dump(test_samples, os.path.join(new_results_dir,'test_samples.sav'))
joblib.dump(test_labels, os.path.join(new_results_dir,'test_labels.sav'))

joblib.dump(validation_samples, os.path.join(new_results_dir,'validation_samples.sav'))
joblib.dump(validation_labels, os.path.join(new_results_dir,'validation_labels.sav'))

In [ ]:
class MyHyperModel(kt.HyperModel):
    
    ## in which stage the parameter is defined e.g, no of units in build, shuffle in fit

    def build(self,hp): # defining a build function with specifics of no of layers, type of layer, optimizer, activation functions
        
            ## creating the search space
        
            LSTM_units = [30, 40] # no of LSTM units
            Drop_rate = [.20, .30, .40] # no of Drop rate
            RecDrop_rate = [.20, .30, .40] # no of Recurrent drop rate
            learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling="log") # learning rate

            model=Sequential()
            model.add(Masking(mask_value=0, input_shape=(4, n_features))),  # n_features = 300
            model.add(LSTM(hp.Choice('LSTM_units',LSTM_units), # choice --> choosing sth from the list               
                           input_shape=(4, n_features), 
                           dropout=hp.Choice('Drop_rate',Drop_rate),
                           recurrent_dropout=hp.Choice('RecDrop_rate', RecDrop_rate)))

            model.add(Dense(45, activation='softmax'))
            model.compile(loss='categorical_crossentropy',optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
                          ,metrics=['accuracy'])

            return model 
        
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            #  whether to shuffle the data in each epoch.
            shuffle=hp.Boolean("shuffle"),
            **kwargs,
        )




In [ ]:
#tuner = kt.GridSearch(
#    MyHyperModel(),
#    objective =kt.Objective("val_loss", direction="min"),
#    overwrite=True,
#    directory=tune_dir ,
#    project_name=project_name,
#)

In [ ]:
# Random search tuner

tuner = kt.RandomSearch(
    MyHyperModel(),
    objective =kt.Objective("val_loss", direction="min"), # optimize for minimizing validation loss
    overwrite=True, # overwrite the previous results in the same directory
    max_trials=30, # max trials to randomly select parameter values (The total number of trials to run during the search)
    directory= tune_dir , 
    project_name=project_name,
)


In [ ]:
tuner.search(train_samples, train_labels, epochs=30, validation_data=(validation_samples, validation_labels),verbose=0)

In [ ]:
# saving the best three trials in a txt file

## Capture the printed output using io.StringIO
summary_output = io.StringIO()

## Use contextlib.redirect_stdout to redirect the print output to the StringIO object
with contextlib.redirect_stdout(summary_output):
    tuner.results_summary(num_trials=3)

## Get the captured output
summary_text = summary_output.getvalue()

file = open(os.path.join(new_results_dir,'best_3params.txt'), 'w')
# Write content to the file
file.write(summary_text)


In [ ]:
# best performing parameter values
best_hp = tuner.get_best_hyperparameters()[0]

In [ ]:
# saving best performing parameter values
joblib.dump(best_hp, os.path.join(new_results_dir,'best_hyperparameters.sav'))

In [ ]:
model = MyHyperModel()

# building the model
best_model = model.build(best_hp)

# training the model
best_model_trained = model.fit(best_hp, best_model, train_samples, 
                               train_labels,validation_data=(validation_samples, validation_labels), epochs=50, verbose=0)

In [ ]:
# remove trial folders from the tunning 

## Define a pattern to match files with names like "trial1", "trial2", etc.

to_remove = os.path.join(tune_dir, project_name)
pattern = "trial*"

## Use glob to find matching files
matching_files = glob.glob(os.path.join(to_remove, pattern))

## Iterate over matching files and delete them
for file_path in matching_files:
    try:
        shutil.rmtree(file_path)
        #print(f"File '{file_path}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting '{file_path}': {e}")

In [ ]:
plt.plot(best_model_trained.history['loss'])
plt.plot(best_model_trained.history['val_loss'])

plt.legend(['train', 'validation'], loc='upper left')

ay = plt.gca()

ay.yaxis.set_major_locator(MultipleLocator(base=0.50)) 


plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

plt.savefig(os.path.join(new_results_dir,'Model_loss.png'))
#plt.show()
plt.close() 

In [ ]:
plt.plot(best_model_trained.history['accuracy'])
plt.plot(best_model_trained.history['val_accuracy'])


ay = plt.gca()
ay.yaxis.set_major_locator(MultipleLocator(base=0.10)) 

plt.legend(['train', 'validation'], loc='upper left')

plt.title('Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')

plt.savefig(os.path.join(new_results_dir,'Model_accuracy.png'))
#plt.show()
plt.close() 

In [ ]:
predictions = best_model.predict(test_samples)

predicted_labels = np.argmax(predictions, axis=1) + 1

true_label = np.argmax(test_labels, axis=1) + 1

In [ ]:
joblib.dump(predictions, os.path.join(new_results_dir,'predictions.sav'))

In [ ]:

cm = confusion_matrix(true_label, predicted_labels, normalize='true')

class_labels = [str(i) for i in range(1, 46)]

plt.figure(figsize=(20, 16))
sns.heatmap(cm, fmt='.2',annot=True, annot_kws={"size": 8}, xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

plt.savefig(os.path.join(new_results_dir,'Cm_optim.png'))
plt.show()
#plt.close() 

In [ ]:
report = classification_report(true_label, predicted_labels, output_dict=True)

In [ ]:
f1 = report['weighted avg']

In [ ]:
file = open(os.path.join(new_results_dir,'F1.txt'), 'w')
# Write content to the file
file.write(str(f1))

In [ ]:
Acc = balanced_accuracy_score(true_label, predicted_labels)
print('Weighted accuracy is:', Acc)

In [ ]:
file = open(os.path.join(new_results_dir,'Acc_balanced.txt'), 'w')
# Write content to the file
file.write(str(Acc))
file.close()

In [ ]:
# Name of the CSV file
csv_filename = 'All_singleElec_accuracies.csv'

# Open the CSV file in append mode
with open(csv_filename, 'a', newline='') as csv_file:
    # Create a CSV writer
    csv_writer = csv.writer(csv_file)
    
    # Write the number to a new row
    csv_writer.writerow([Acc])
    
csv_file.close()